In [1]:
import sys
sys.path.append("/hdd1/user19/bag/2.fastText/src")
import time
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data import Dictionary, CustumDataset
from model import SISG
from utils import *
from itertools import islice

import numpy as np
from scipy import stats
import pickle

In [2]:
lang = "de"

In [3]:
import pickle

file_path = f"../src/{lang}_dictionary.pkl"
with open(file_path,"rb") as file:
    dictionaries = pickle.load(file)

In [4]:
len(dictionaries.ids)

2668582

In [5]:
model_path = "input your path"
device = "cpu"
model_info = torch.load(model_path, map_location=device)
best_model = model_info['model']
best_model.load_state_dict(model_info['model_state_dict'])

<All keys matched successfully>

---

In [18]:
w1, w2, base_corr = load_zg222()
vocab_size = len(dictionaries.ids)

In [26]:
ngram = [3,4,5,6]
oov = 0
corr_list = []
b_corr_list = []
with torch.no_grad():
    for word1, word2, bc in zip(w1, w2, base_corr):
        word1, word2 = word1.lower(), word2.lower()
        if word1 in dictionaries.word2id.keys():
            word1_id = torch.LongTensor([dictionaries.word2id[word1]]).to(device)
            word1_subword = list(map(lambda x: (x+vocab_size)%MAX_SUBWORD_VOCAB_SIZE, dictionaries.word2subword[word1_id.item()]))
            word1_id %= MAX_SUBWORD_VOCAB_SIZE
            if len(word1_subword) == 1:
                word1_e = best_model.word_in_emb(word1_id).view(-1).to(device=device)
                l2norm = word1_e.pow(2).sum().sqrt().to(device=device)
                word1_emb = word1_e/(l2norm)
            else:
                word1_subword = torch.LongTensor(word1_subword).to(device)
                word1_subword_emb = best_model.word_in_emb(word1_subword).mean(dim=0)
                word1_emb = best_model.word_in_emb(word1_id).view(-1)
                word1_emb += word1_subword_emb
                l2_norm = word1_emb.pow(2).sum().sqrt()
                word1_emb /= (l2_norm)
        else:
            word1_emb = torch.zeros(300).to(device)
            oov += 1
            # continue
            word1_subword = get_subword(word1, ngram, dictionaries)
            word1_subword = list(map(lambda x: (x+vocab_size)%MAX_SUBWORD_VOCAB_SIZE, word1_subword))
            word1_subword = torch.LongTensor(word1_subword).to(device)
            word1_emb = best_model.word_in_emb(word1_subword).mean(dim=0)
            l2_norm = word1_emb.pow(2).sum().sqrt()
            word1_emb = word1_emb/(l2_norm + 1e-7)
            
        if word2 in dictionaries.word2id.keys():
            word2_id = torch.LongTensor([dictionaries.word2id[word2]]).to(device)
            word2_subword = list(map(lambda x: (x+vocab_size)%MAX_SUBWORD_VOCAB_SIZE, dictionaries.word2subword[word2_id.item()]))
            word2_id %= MAX_SUBWORD_VOCAB_SIZE
            if len(word2_subword) == 1:
                word2_e = best_model.word_in_emb(word2_id).view(-1).to(device=device)
                l2norm = word2_e.pow(2).sum().sqrt().to(device=device)
                word2_emb = word2_e/(l2norm)
            else:
                word2_subword = torch.LongTensor(word2_subword).to(device)
                word2_subword_emb = best_model.word_in_emb(word2_subword).mean(dim=0)
                word2_emb = best_model.word_in_emb(word2_id).view(-1)
                word2_emb += word2_subword_emb
                l2_norm = word2_emb.pow(2).sum().sqrt()
                word2_emb /= (l2_norm)
        else:
            word2_emb = torch.zeros(300).to(device)
            oov += 1
            # continue
            word2_subword = get_subword(word2, ngram, dictionaries)
            word2_subword = list(map(lambda x: (x+vocab_size)%MAX_SUBWORD_VOCAB_SIZE, word2_subword))
            word2_subword = torch.LongTensor(word2_subword).to(device)
            word2_emb = best_model.word_in_emb(word2_subword).mean(dim=0)
            l2_norm = word2_emb.pow(2).sum().sqrt()
            word2_emb = word2_emb/(l2_norm + 1e-7)
        corr_list.append(torch.sum(word1_emb* word2_emb).item())
        b_corr_list.append(bc)

In [27]:
b_corr_list = np.array(list(map(float,b_corr_list)))

In [28]:
l2_norm = np.sqrt(np.sum(np.power(b_corr_list,2)))
b_corr_list /= (l2_norm + 1e-7)

In [29]:
oov

19

In [30]:
corr = stats.spearmanr(corr_list, b_corr_list)[0]
print(corr)

0.047533804937815824


In [ ]:
from args import *
from tqdm import tqdm

vocab_num = len(dictionaries.ids)

word_vec = torch.zeros((len(dictionaries.ids),300)).to(device=device)
with torch.no_grad():
    best_model.eval()
    for iii in tqdm(dictionaries.ids):
        subword_id = list(map(lambda x: (x+vocab_num)%MAX_SUBWORD_VOCAB_SIZE, dictionaries.word2subword[iii]))
        # subword_id = subword_id
        if len(subword_id) == 0:
            word_e = best_model.word_in_emb(torch.LongTensor([iii])).view(-1).to(device=device)
            l2norm = word_e.pow(2).sum().sqrt().to(device=device)
            word_vec[iii] = word_e/l2norm
            continue
        subword_id = torch.LongTensor(subword_id).to(device=device)
        # subword_id = torch.LongTensor(subword_id + [iii]).to(device=device)
        si = best_model.word_in_emb(subword_id).to(device=device)
        si = si.mean(dim=0).to(device=device)
        si += best_model.word_in_emb(torch.LongTensor([iii])).view(-1).to(device=device)
        l2norm = si.pow(2).sum().sqrt().to(device=device) #L2Norm
        word_vec[iii] = si/l2norm

In [ ]:
def test_words(path = "../questions-words.txt"):
    with open(path, 'r', encoding = "UTF8") as f:
        temp = f.readlines()

    semantic_words = []
    syntatic_words = []
    for e in temp:
        t = e[:-1].split(" ")
        if t[1] == "gram2-opposite":
        # if t[1] == "gram1-adjective-to-adverb":
            break
        if t[0] == ":":
            continue
        words = [tt.lower() for tt in t]
        semantic_words.append(words)
    for e in temp[::-1]:
        t = e[:-1].split(" ")
        if t[1] == "gram2-opposite":
        # if t[1] == "gram1-adjective-to-adverb":
            break
        if t[0] == ":":
            continue
        words = [tt.lower() for tt in t]
        syntatic_words.append(words)

    return np.array(semantic_words), np.array(syntatic_words)

In [ ]:
sem, syn = test_words()

In [ ]:
sem_size = 0
sem_score = 0
for anal_words in tqdm(sem):
    if anal_words[0] not in dictionaries.word2id:
        continue
    if anal_words[1] not in dictionaries.word2id:
        continue
    if anal_words[2] not in dictionaries.word2id:
        continue
    if anal_words[3] not in dictionaries.word2id:
        continue
    sem_size += 1
    
    word1, word2, word3, word4 = list(map(lambda x:dictionaries.word2id[x], anal_words))
    query = word_vec[word2] - word_vec[word1] + word_vec[word3]
    similarities = torch.matmul(query, word_vec.T)
    similarities[[word1, word2, word3]] = 0
    predict = torch.topk(similarities, k=1)[1]
    if word4 in predict:
        sem_score += 1

In [ ]:
syn_size = 0
syn_score = 0
for anal_words in tqdm(syn):
    if anal_words[0] not in dictionaries.word2id:
        continue
    if anal_words[1] not in dictionaries.word2id:
        continue
    if anal_words[2] not in dictionaries.word2id:
        continue
    if anal_words[3] not in dictionaries.word2id:
        continue
    syn_size += 1
    
    word1, word2, word3, word4 = list(map(lambda x:dictionaries.word2id[x], anal_words))
    query = word_vec[word2] - word_vec[word1] + word_vec[word3]
    similarities = torch.matmul(query, word_vec.T)
    similarities[[word1, word2, word3]] = 0
    predict = torch.topk(similarities, k=1)[1]
    if word4 in predict:
        syn_score += 1

In [ ]:
sem_score/sem_size

In [ ]:
syn_score/syn_size

In [ ]:
lang = "de"
file_path = f"../{lang}_dictionary_v0.pkl"
with open(file_path,"rb") as file:
    dictionaries = pickle.load(file)
model_path = f"../sisg_{lang}_shuffle_v0.pth"
device = "cpu"
model_info = torch.load(model_path, map_location=device)
best_model = model_info['model']
best_model.load_state_dict(model_info['model_state_dict'])

vocab_num = len(dictionaries.ids)

word_vec = torch.zeros((len(dictionaries.ids),300)).to(device=device)

with torch.no_grad():
    best_model.eval()
    for iii in tqdm(dictionaries.ids):
        subword_id = list(map(lambda x: (x+vocab_num)%MAX_SUBWORD_VOCAB_SIZE, dictionaries.word2subword[iii]))
        # subword_id = subword_id
        if len(subword_id) == 0:
            word_e = best_model.word_in_emb(torch.LongTensor([iii])).view(-1).to(device=device)
            l2norm = word_e.pow(2).sum().sqrt().to(device=device)
            word_vec[iii] = word_e/l2norm
            continue
        subword_id = torch.LongTensor(subword_id).to(device=device)
        # subword_id = torch.LongTensor(subword_id + [iii]).to(device=device)
        si = best_model.word_in_emb(subword_id).to(device=device)
        si = si.mean(dim=0).to(device=device)
        si += best_model.word_in_emb(torch.LongTensor([iii])).view(-1).to(device=device)
        l2norm = si.pow(2).sum().sqrt().to(device=device) #L2Norm
        word_vec[iii] = si/l2norm
        
sem, syn = test_words(path = "../de_trans_Google_analogies.txt")

sem_size = 0
sem_score = 0
for anal_words in tqdm(sem):
    if anal_words[0] not in dictionaries.word2id:
        continue
    if anal_words[1] not in dictionaries.word2id:
        continue
    if anal_words[2] not in dictionaries.word2id:
        continue
    if anal_words[3] not in dictionaries.word2id:
        continue
    sem_size += 1
    
    word1, word2, word3, word4 = list(map(lambda x:dictionaries.word2id[x], anal_words))
    query = word_vec[word2] - word_vec[word1] + word_vec[word3]
    similarities = torch.matmul(query, word_vec.T)
    similarities[[word1, word2, word3]] = 0
    predict = torch.topk(similarities, k=1)[1]
    if word4 in predict:
        sem_score += 1
        
syn_size = 0
syn_score = 0
for anal_words in tqdm(syn):
    if anal_words[0] not in dictionaries.word2id:
        continue
    if anal_words[1] not in dictionaries.word2id:
        continue
    if anal_words[2] not in dictionaries.word2id:
        continue
    if anal_words[3] not in dictionaries.word2id:
        continue
    syn_size += 1
    
    word1, word2, word3, word4 = list(map(lambda x:dictionaries.word2id[x], anal_words))
    query = word_vec[word2] - word_vec[word1] + word_vec[word3]
    similarities = torch.matmul(query, word_vec.T)
    similarities[[word1, word2, word3]] = 0
    predict = torch.topk(similarities, k=1)[1]
    if word4 in predict:
        syn_score += 1

In [ ]:
sem_score/sem_size

In [ ]:
syn_score/syn_size